In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import importlib
import causal_examples
from sklearn.inspection import PartialDependenceDisplay
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from econml.dml import CausalForestDML
from econml.cate_interpreter import SingleTreeCateInterpreter
import matplotlib.pyplot as plt

# importlib.reload(causal_examples)

rng = np.random.default_rng()

/Users/raphaeltamaki/Documents/personal_git/causal_venv/lib/python3.9/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/Users/raphaeltamaki/Documents/personal_git/causal_venv/lib/python3.9/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-

/Users/raphaeltamaki/Documents/personal_git/causal_venv/lib/python3.9/site-packages/shap/explainers/_partition.py:676: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def lower_credit(i, value, M, values, clustering):
The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is cu

### Drug -> Blood Pressure -> Heart Attack

In [2]:
data = causal_examples.BloodPressureExample().generate_effects()
X = data[['took_drug', 'low_blood_pressure']].copy()
y = data['heart_attack']
# X = data[['took_drug']].copy()
# y = data['low_blood_pressure']
data.head()

,took_drug,low_blood_pressure,heart_attack
0,1,1,0
1,1,1,0
2,0,1,0
3,1,1,0
4,0,0,0


### Linear Regression

In [3]:
reg = LinearRegression()
reg.fit(X, y)
reg.coef_

array([-0.00088735, -0.00420062])

### Decision Tree

In [4]:
tree = DecisionTreeRegressor()
tree.fit(X, y)
r2_score(tree.predict(X), y)
tree.feature_importances_

array([0.03823242, 0.96176758])

In [5]:
PartialDependenceDisplay.from_estimator(tree, X, ['took_drug'])

### Causal Tree

In [6]:
causal_model = CausalForestDML(n_estimators=1, subforest_size=1, inference=False, max_depth=4)
causal_model = causal_model.fit(Y=y, X=X, T=data['took_drug'])

In [7]:
intrp = SingleTreeCateInterpreter(max_depth=2).interpret(causal_model, X)
intrp.plot(feature_names=X.columns, fontsize=12)

In [8]:
np.mean(causal_model.ate(X))

712.6537463785409

## Discount Ticket

In [9]:
importlib.reload(causal_examples)
data = causal_examples.DiscountTicket().generate_effects()
X = data[['income', 'ticket']].copy()
y = data['spend']

data.head()

,income,ticket,spend
0,1361.189186,0.000000,472.237837
1,1340.725844,0.000000,468.145169
2,841.318675,0.000000,368.263735
3,704.762148,0.141731,369.298693
4,972.739205,0.000000,394.547841


### Linear Regression

In [10]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X, y)
reg.coef_

array([  0.2, 200. ])

In [11]:
PartialDependenceDisplay.from_estimator(tree, X, ['ticket'])

Attempting to set identical low and high ylims makes transformation singular; automatically expanding.


### Decision Tree

In [12]:

tree = DecisionTreeRegressor()
tree.fit(X, y)
r2_score(tree.predict(X), y)
tree.feature_importances_

array([0.91549573, 0.08450427])

In [13]:
PartialDependenceDisplay.from_estimator(tree, X, ['ticket'])

### Causal Tree

In [14]:
causal_model = CausalForestDML(n_estimators=1, subforest_size=1, inference=False, max_depth=4)
causal_model = causal_model.fit(Y=y, X=X, T=data['ticket'])

In [15]:
intrp = SingleTreeCateInterpreter(max_depth=2).interpret(causal_model, X)
intrp.plot(feature_names=X.columns, fontsize=12)

In [16]:
np.mean(causal_model.ate(X))

200.63834877142006

In [17]:
causal_model.add_treatment_effect(X.copy())

AttributeError: 'CausalForestDML' object has no attribute 'add_treatment_effect'

In [36]:
from ipywidgets import interact, interactive, fixed, VBox, Output
import ipywidgets as widgets

# Define the initial widgets
first_widget = widgets.Dropdown(
    options=['Option 1', 'Option 2', 'Option 3'],
    description='First Widget:'
)

second_widget = widgets.IntSlider(
    min=0, max=10, step=1, description='Second Widget:'
)

third_widget = widgets.Text(
    description='Third Widget:'
)

output = Output()  # Create an Output widget for displaying the widget container

def update_widgets(change):
    # Get the selected value of the first widget
    selected_value = first_widget.value

    # Clear the output area and display the updated widget container
    with output:
        output.clear_output()
        if selected_value == 'Option 1':
            display(VBox([second_widget]))
        elif selected_value == 'Option 2':
            display(VBox([third_widget]))

# Register the callback function to the first widget's `observe` method
first_widget.observe(update_widgets, 'value')

# Display the initial widgets
display(first_widget)
display(output)  # Display the output widget for displaying the widget container


Dropdown(description='First Widget:', options=('Option 1', 'Option 2', 'Option 3'), value='Option 1')

Output()

In [37]:
?widgets.Dropdown